In [138]:
from pydub import AudioSegment
from pydub.generators import Sine
import math

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb):
    """Convert an RGB color to hexadecimal format."""
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

def color_distance(rgb1, rgb2):
    """Calculate the Euclidean distance between two RGB colors."""
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(rgb1, rgb2)))

def closest_color(rgb, color_map):
    """Find the closest color in the color_map to the given RGB color."""
    closest_colors = sorted(color_map.keys(), key=lambda color: color_distance(rgb, hex_to_rgb(color)))
    return closest_colors[0]

In [139]:
color_to_freq_map = {
    '#FF0000': 261.63,  # C - Red
    '#FFA500': 392.00,  # G - Orange-pink
    '#FFFF00': 293.66,  # D - Yellow
    '#008000': 440.00,  # A - Green
    '#ADD8E6': 329.63,  # E - Whitish-blue
    '#87CEEB': 493.88,  # B - Similar to E
    '#0000FF': 369.99,  # F♯ - Blue, bright
    '#EE82EE': 277.18,  # D♭ - Violet
    '#800080': 415.30,  # A♭ - Purplish-violet
    '#B0C4DE': 311.13,  # E♭ - Steel color with metallic sheen
    '#5F9EA0': 466.16,  # B♭ - Similar to E flat
    '#8B0000': 349.23,  # F - Red, dark
}



In [140]:
def image_to_music(main_colors, pixels, note_duration=1000):
    # Define the minimum length as a percentage of the note_duration
    min_length_percentage = 0.05
    min_note_length = note_duration * min_length_percentage

    song = AudioSegment.silent(duration=0)  # Start with a silent segment

    for row in pixels:
        row_length = len(row)
        current_color = main_colors[row[0]]
        current_count = 1

        note_groups = []

        # Iterate over the row to group colors
        for idx in row[1:]:
            hex_color = main_colors[idx]
            if hex_color == current_color:
                current_count += 1
            else:
                # Process the previous group
                note_groups.append((current_color, current_count))
                current_color = hex_color
                current_count = 1
        # Add the last color group
        note_groups.append((current_color, current_count))

        # Calculate the total length for notes that are above the minimum length
        total_above_min = sum(count for color, count in note_groups if count / row_length >= min_length_percentage)

        # Generate notes
        for hex_color, count in note_groups:
            proportion = count / row_length
            if proportion < min_length_percentage:
                note_time = min_note_length
            else:
                note_time = note_duration * (count / total_above_min)

            rgb_color = hex_to_rgb(hex_color)
            closest_hex = closest_color(rgb_color, color_to_freq_map)
            freq = color_to_freq_map[closest_hex]
            note = Sine(freq).to_audio_segment(duration=int(note_time))
            song += note

    output_file = "output_song.wav"
    song.export(output_file, format="wav")

    print("Generated music saved to:", output_file)
    return output_file  # To allow accessing the output location


In [141]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans

def load_and_cluster(path, n_clusters, display=False):
    image = Image.open(path)
    x, y = image.size
    
    h = 150 / float(y)

    redim = min(x, h)

    new_h = int(y * redim)

    image = image.resize((x, new_h), Image.ANTIALIAS)

    if image.mode != 'RGB':
        image = image.convert('RGB')

    pixels = list(image.getdata())
    pixels_list = [pixels[i * x:(i + 1) * x] for i in range(y)]
    image_array = np.array(image)
    pixels = image_array.reshape(-1, 3)

    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(pixels)
    labels = kmeans.labels_

    for i in range(y):
        for j in range(x):
            pixel_index = i * x + j
            pixels_list[i][j] = labels[pixel_index]
    centers = [rgb_to_hex(i) for i in kmeans.cluster_centers_.astype(int)]
    return centers, pixels_list

In [142]:
main_colors, pixels_list = load_and_cluster("Flag_of_France.png", 3)
print(main_colors)

['#ed2939', '#fefefe', '#002395']


In [143]:
image_to_music(main_colors, pixels_list)

Generated music saved to: output_song.wav


'output_song.wav'